In [3]:
import sys
import os
import torch
import glob
import pandas as pd
import numpy as np
import pingouin as pg
from age_prediction.models.\
    efficientnet_pytorch_3d import EfficientNet3D as EfNetB0
from age_prediction.metrics import MSE, MAE
from age_prediction.trainer import ModuleTrainer
from age_prediction.dataloader import MyDataLoader

In [4]:
def check_state_dict(ck, md):
    if ck.keys() == md.keys():
        return ''
    else:
        split_key = list(ck.keys())[0].split(
            list(md.keys())[0])[0]
        return split_key

def correct_state_dict(ck, split_key):
    ck2 = {}
    for k in ck.keys():
        ck2[k.split(split_key)[1]] = ck[k]
    return ck2

def MAE_pred(y_pred, y_true):
    mae = torch.nn.L1Loss(reduction='mean')(y_pred, y_true).cpu().detach().numpy().item()
    return mae

def MSE_pred(y_pred, y_true):
    mse = torch.nn.MSELoss()(y_pred, y_true).cpu().detach().numpy().item()
    return mse

def delta_pred(y_pred, y_true):
    delta = y_pred - y_true
    return np.mean(delta)

def pearson(y_pred, y_true):
    return pg.corr(y_pred, y_true)['r'][0]


In [85]:
def get_snapshot_results(side, snapshot, testfile, dropout_rate=0.2):
    # Load effNet3D B0
    model = EfNetB0.from_name("efficientnet-b0",
                              override_params={
                                'num_classes': 1,
                                'dropout_rate': dropout_rate
                              },
                              in_channels=1,
                              )
    device = torch.device('cpu')
    model = model.to(device)

    optimizer = torch.optim.RMSprop(model.parameters(),
                                    lr=.256, alpha=0.9,
                                    eps=1e-08, momentum=0.9,
                                    weight_decay=0)

    # print('Loading model from {}'.format(snapshot))
    checkpoint = torch.load(snapshot, map_location=device)
    split_key = check_state_dict(checkpoint['state_dict'], model.state_dict())
    if split_key is not None:
        checkpoint['state_dict'] = correct_state_dict(checkpoint['state_dict'], split_key)

    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    epoch = checkpoint['epoch']
    # _loss = checkpoint['loss']
    # _val_loss = checkpoint['val_loss']
    # print("Snapshot trained for {} epochs. \
    # Loss: {} and Val loss {}".format(epoch, _loss, _val_loss))

    loss = torch.nn.L1Loss(reduction='mean')
    metrics = [MSE()]
    
    # Predict
    trainer = ModuleTrainer(model.to(device))

    trainer.compile(loss=loss,
                    optimizer=optimizer,
                    metrics=metrics)
    
    dataloader = MyDataLoader(database='../datasets',
                              csv_data='database_split',
                              side=side,
                              batch=512,
                              data_aug=False,
                              test_file=testfile
                              )
    dataloader.prepare_data('test_label')
    dataloader.setup('test_label')
    
    # print("Test size", len(dataloader.test.inputs[0]))


    pred = trainer.predict_loader(dataloader.testlabel_dataloader(),
                                  cuda_device=False)
    
    imgs = dataloader.testpath
    imgs = [img.split("/")[-1].split(side)[0] for img in imgs]
    
    preds = pd.concat([pd.DataFrame(imgs),
                       pd.DataFrame(dataloader.testlabel),
                       pd.DataFrame(pred.detach().cpu().numpy())], axis=1)
    preds.columns = ['Name', 'True', 'Pred']
    
    return preds, epoch

def metrics_pipeline(preds):
    mae = MAE_pred(torch.from_numpy(preds['Pred'].values),
                   torch.from_numpy(preds['True'].values))
    mse = MSE_pred(torch.from_numpy(preds['Pred'].values),
                   torch.from_numpy(preds['True'].values))
    rmse = np.sqrt(mse)
    delta = delta_pred(preds['Pred'],
                       preds['True'])
    r = pearson(preds['Pred'],
                preds['True'])
    return mae, rmse, delta, r


In [89]:
def organize_results(res, type, dp, ep=None, age=None, side=None):
    if age is not None:
        df = {'side': side, 'age': age}
        if ep is not None:
            df['epoch'] = ep
        df['mae_' + type] = res[0]
        df['rmse_' + type] = res[1]
    else:
        df = {'mae_' + type: res[0], 'rmse_' + type: res[1]}
    if type != 'train':
        _, _, delta, r = res
        df['delta_' + type] = delta
        df['r_' + type] = r   

    return pd.DataFrame(df, index=[dp])


In [90]:
def get_results(side, age):
    results = pd.DataFrame()
    for ckpt in glob.glob('weights/ckpt_*'):
        if age in ckpt and side in ckpt:
            if age=='0-70' and side == '_L':
                dropout_rate=0.3
            else:
                dropout_rate=0.2
            # val
            pred, epoch = get_snapshot_results(side, ckpt, 'val_'+age+'.csv', dropout_rate)
            # print(pred)
            val = organize_results(metrics_pipeline(pred), 'val', dropout_rate)
            # print("TEST")
            pred, epoch = get_snapshot_results(side, ckpt, 'test_'+age+'.csv', dropout_rate)
            test = organize_results(metrics_pipeline(pred), 'test', dropout_rate)
            df = pd.concat([val, test], axis=1)
            results = pd.concat([results, df])
    return results.sort_index()


In [91]:
res_R = get_results('_L', '0-70')
res_L = get_results('_R', '0-70')

results = pd.concat([res_L, res_R])
results

,mae_val,rmse_val,delta_val,r_val,mae_test,rmse_test,delta_test,r_test
0.2,4.239758,5.810155,-0.038412,0.923164,4.710463,6.108817,-1.536577,0.925940
0.3,4.801819,6.289953,0.489273,0.915197,4.235512,6.027559,0.173140,0.927498


In [92]:
results.to_csv('predict_results/metrics_0-70.csv')

In [94]:
res_R = get_results('_L', '70-100')
res_L = get_results('_R', '70-100')

results = pd.concat([res_L, res_R])
results, results.mean()